# Script to Organize Raw MavLink logs 
### Our methodology to organize the raw MavLink log was to first segregate the data based on the datatype defined in column J
### We separated the data by putting each row correspoding to the datatype into a separate dataframe and reorganizing it by placing the column headers in the proper places with corresponding values

### We also added preprocessing to add numerical column values to the dataframe for easier analysis as the raw Mavlink logs do not have column headers 

### We choose to go with excel filetype as the CSV files mess up the timestamps

## To use the script, simply download it where you want to save the outputted logs, run it and input the path for the raw mavlink log

With love from Spruha, Thomas, and Josh

In [22]:
# preprocess excel file to add column labels
import os
import sys
import openpyxl
from openpyxl import load_workbook

file_path = input("Enter path to mavlink log: ")
if (type(file_path) != str or file_path.strip() == ""):
    raise ValueError("Invalid path")

if (not os.path.isfile(file_path)):
    raise ValueError("File '" + file_path + "' does not exist")

# Done after to so it doesn't error out on a nonstring type
file_path = file_path.strip()
print(file_path)

excel_file = file_path
# Load the existing Excel file
wb = load_workbook(excel_file)

# Select the worksheet where you want to add the column headers
sheet = wb.active

# Get the headers from the first row of the worksheet
headers = [cell.value for cell in sheet[1]]

# Check if the headers are numerical from 0 to the last column
numerical_headers = all(str(i) == header for i, header in enumerate(headers))

if not numerical_headers:
    print("No numerical column headers found, adding numerical column headers")
    # created numbered column headers up to the last active column
    last_column = sheet.max_column

    print("last column: ", last_column)

    column_headers = []

    for i in range(0, last_column+1):
        column_headers.append(str(i))

    print(column_headers)

    # Insert a new row at the top of the worksheet and write the column headers
    sheet.insert_rows(1)
    for col, header in enumerate(column_headers, 1):
        sheet.cell(row=1, column=col, value=header)
else:
    print("Numerical Column headers found, no need to add headers")
# Save the modified Excel file
wb.save(excel_file)



Test-log.xlsx


In [23]:
import pandas as pd

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path)
df = pd.DataFrame(df)
df.columns = df.columns.astype(str)

print(df.head(100))

                          0   1   2  3  4   5   6   7       8  \
0   2023-05-02T18:07:06.638  FD  20  0  0  28  FF  BE      4C   
1   2023-05-02T18:07:06.638  FD  20  0  0  30  FF  BE      4C   
2   2023-05-02T18:07:06.689  FD   9  0  0  31  FF  BE       0   
3   2023-05-02T18:07:06.692  FD   9  0  0   1   1   1       0   
4   2023-05-02T18:07:06.703  FD   9  0  0  5E  FF  BE       0   
..                      ...  ..  .. .. ..  ..  ..  ..     ...   
95  2023-05-02T18:07:11.091  FD  1C  0  0  49   1   1      21   
96  2023-05-02T18:07:11.091  FD   A  0  0  4A   1   1       2   
97  2023-05-02T18:07:11.133  FD   6  0  0  2A  FF  BE      42   
98  2023-05-02T18:07:11.138  FD  1C  0  0  4B   1   1      A3   
99  2023-05-02T18:07:11.138  FD   2  0  0  4C   1   1      A5   

                                9  ...  49   50  51  52   53   54  55   56  \
0          mavlink_command_long_t  ... NaN  NaN NaN NaN  NaN  NaN NaN  NaN   
1          mavlink_command_long_t  ... NaN  NaN NaN NaN  NaN  N

In [24]:
# Get all the different types of data
categories = df['9'].unique()
print(categories.size)


30


In [25]:
# create separate dataframes for each type specified in the 9th column (column J from the raw csv file)
new_logframes = {}

for category in categories:
    new_logframes[category] = df[df['9'] == category]
    print (category)
    print (new_logframes[category].head(10))



mavlink_command_long_t
                          0   1   2  3  4   5   6   7       8  \
0   2023-05-02T18:07:06.638  FD  20  0  0  28  FF  BE      4C   
1   2023-05-02T18:07:06.638  FD  20  0  0  30  FF  BE      4C   
8   2023-05-02T18:07:06.877  FD  20  0  0  32  FF  BE      4C   
13  2023-05-02T18:07:07.147  FD  20  0  0  33  FF  BE      4C   

                         9  ...  49   50  51  52   53   54  55   56  57  58  
0   mavlink_command_long_t  ... NaN  NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  
1   mavlink_command_long_t  ... NaN  NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  
8   mavlink_command_long_t  ... NaN  NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  
13  mavlink_command_long_t  ... NaN  NaN NaN NaN  NaN  NaN NaN  NaN NaN NaN  

[4 rows x 59 columns]
mavlink_heartbeat_t
                          0   1  2  3  4   5   6   7  8                    9  \
2   2023-05-02T18:07:06.689  FD  9  0  0  31  FF  BE  0  mavlink_heartbeat_t   
3   2023-05-02T18:07:06.692  FD  9  0  0   1   1   1  0  m

In [26]:
import math
final_dataframes = {}
file_names = []

for category in new_logframes:
    
    col_names = ['Time']
    col = 10

    # get all column names from new_logframes
    while col < 60:
    #    create new blank column in final_dataframes column based on the name from new_logframes
        col_name = new_logframes[category].iloc[0,col]
        if (type(col_name) == str):
            col_names.append(col_name)
        col+=2


    # add category names into an array to be used as file names
    file_names.append(category)

    # drop column 9
    new_logframes[category] = new_logframes[category].drop(columns = ['9'])
    
    # drop all paramter columns
    col = 10
    while col < 60:
        new_logframes[category] = new_logframes[category].drop(str(col), axis=1)
        col+=2
    
    # drop all byte data
    new_logframes[category] = new_logframes[category].drop(new_logframes[category].columns[1:9], axis=1)

    print(col_names)
    print(len(col_names))

    # drop all rows past length of col_names
    new_logframes[category] = new_logframes[category].iloc[:, 0:len(col_names)]

    # rename column names
    new_logframes[category].columns = col_names

    print(new_logframes[category].head(10))

    

['Time', 'param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'param7', 'command', 'target_system', 'target_component', 'confirmation', 'Len', 'crc16']
14
                       Time  param1   param2 param3 param4 param5 param6  \
0   2023-05-02T18:07:06.638   148.0        0      0      0      0      0   
1   2023-05-02T18:07:06.638   242.0  1000000      0      0      0      0   
8   2023-05-02T18:07:06.877   148.0        0      0      0      0      0   
13  2023-05-02T18:07:07.147   395.0        0      0      0      0      0   

   param7 command target_system target_component confirmation   Len crc16  
0       0     512             1                1            0  sig     44  
1       0     511             1                1            0  sig     44  
8       0     512             1                1            0  sig     44  
13      0     512             1                1            0  sig     44  
['Time', 'custom_mode', 'type', 'autopilot', 'base_mode', 'system_status', 'm

In [27]:
# output all dataframes in new_logframes to csv files


# create new output directory
from pathlib import Path

# Define the name of the new directory
new_directory_name = file_path + "-organized-UAV-Logs"

# Create the directory using Path.mkdir()
new_directory = Path(new_directory_name)
new_directory.mkdir(exist_ok=True)  # exist_ok=True allows it to continue if the directory already exists

print(f"Directory '{new_directory_name}' created successfully.")

for category in new_logframes:
    new_logframes[category].to_excel(new_directory_name + '/' + category + '.xlsx', index=False)

Directory 'Test-log.xlsx-organized-UAV-Logs' created successfully.
